# Import, options

In [1]:
import numpy, matplotlib, pandas, scanpy

In [2]:
matplotlib.rcParams.update({'font.size':20, 'xtick.labelsize':20, 'ytick.labelsize':20, 
                            'axes.grid' : True, 'grid.alpha': 0.5, 'grid.linestyle' : ':',
                            'figure.figsize':(8, 5), 'svg.fonttype' : 'none'})

In [3]:
file_directory = "/Users/kja11/OneDrive - Menntaský/PostDoc_Hypothermia/in_silico/Python/"

# Data Downloads

In [6]:
#data mouse_NPC
mouse_NPC_df = pandas.read_csv(file_directory+'1) input/mNPC_all_detected_genes_DEseq.csv',
                               sep = ',')
print(mouse_NPC_df.shape, '\n', mouse_NPC_df.columns)

mouse_NPC_df = mouse_NPC_df[['gene', '32_degrees', '32_degrees.1', '32_degrees.2', '32_degrees.3']]
mouse_NPC_df

(16062, 15) 
 Index(['Row.names', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue',
       'padj', 'gene', '37_degrees', '37_degrees.1', '37_degrees.2',
       '32_degrees', '32_degrees.1', '32_degrees.2', '32_degrees.3'],
      dtype='object')


,gene,32_degrees,32_degrees.1,32_degrees.2,32_degrees.3
0,Cirbp,6117.014966,5155.322063,5886.014969,5396.949830
1,Xbp1,683.013572,668.546151,700.131803,612.231119
2,Hspa8,46940.696119,43593.055672,45443.791959,41847.851131
3,Dnaja3,1225.709230,1163.293883,1185.274805,1204.699735
4,Tra2a,652.603893,620.523550,590.851333,453.756987
...,...,...,...,...,...
16057,Pex2,1629.899000,1835.107000,1833.215000,1462.453000
16058,Ank3,990.711700,794.611200,910.848200,715.005000
16059,Hmgcl,1115.568000,995.605900,1094.933000,1095.132000
16060,Arl6ip5,1355.401000,1512.990000,1346.521000,1217.732000


In [7]:
# DL List of the SP1 candidat repressors
SP1_repressors = pandas.read_csv(file_directory+'1) input/SP1_repressors.csv', sep = ',')
SP1_repressors = SP1_repressors[['Gene Symbol']]
SP1_repressors

,Gene Symbol
0,STXBP1
1,LRRC26
2,MUC15
3,GATAD2A
4,MUT
...,...
490,CNTNAP2
491,hsa-mir-5693
492,hsa-mir-300
493,ZNF816-ZNF321P


In [8]:
# DL list human and mouse orthologs
list_hum_mous = pandas.read_csv(file_directory+'human_mouse_1to1_orthologs.csv', sep = ',')
list_hum_mous

,human,mouse
0,A1BG,A1bg
1,A1CF,A1cf
2,A2LD1,A2ld1
3,A2M,A2m
4,A4GALT,A4galt
...,...,...
15542,ZYG11A,Zyg11a
15543,ZYG11B,Zyg11b
15544,ZYX,Zyx
15545,ZZEF1,Zzef1


# Transform to have mice gene symbols 

In [9]:
#rename to be able to merge the two df
SP1_repressors = SP1_repressors.rename(columns = {'Gene Symbol' : 'human'})

# merge the SP1 df with the hum_mouse to have mouse gene symbols
SP1_repressors = SP1_repressors.merge(list_hum_mous)
print(SP1_repressors)

SP1_repressors = SP1_repressors[['mouse']]
SP1_repressors = SP1_repressors.rename(columns = {'mouse' : 'gene'})
SP1_repressors

       human     mouse
0     STXBP1    Stxbp1
1     LRRC26    Lrrc26
2      MUC15     Muc15
3    GATAD2A   Gatad2a
4        MUT       Mut
..       ...       ...
324    WNT11     Wnt11
325   R3HDM1    R3hdm1
326     RGMB      Rgmb
327  C9orf78  BC005624
328  CNTNAP2   Cntnap2

[329 rows x 2 columns]


,gene
0,Stxbp1
1,Lrrc26
2,Muc15
3,Gatad2a
4,Mut
...,...
324,Wnt11
325,R3hdm1
326,Rgmb
327,BC005624


In [11]:
# Merge the two tables and only keep the sp1_repressors in the RNAseq
df_mNPC_sp1rep = pandas.merge(mouse_NPC_df, SP1_repressors, on= 'gene')

print(f'number of Sp1 repressors in the RNAseq is {len(df_mNPC_sp1rep)} on a total of {len(SP1_repressors)}')
print(f'{len(SP1_repressors) - len(df_mNPC_sp1rep)} repressors are not in the RNAseq')

df_mNPC_sp1rep.head()

number of Sp1 repressors in the RNAseq is 267 on a total of 329
62 repressors are not in the RNAseq


,gene,32_degrees,32_degrees.1,32_degrees.2,32_degrees.3
0,Creld2,751.602695,748.099155,778.773074,801.580112
1,Tfdp2,3349.830478,3247.058555,3030.374898,3307.762556
2,Noc3l,827.786899,853.633273,754.984167,665.868631
3,Pias3,1454.152781,1520.922170,1563.303656,1484.663052
4,Mns1,975.424977,783.570507,944.911361,746.454093


In [13]:
#control values did not change.
print(mouse_NPC_df[mouse_NPC_df['gene'] == 'Tfdp2'])
print(df_mNPC_sp1rep[df_mNPC_sp1rep['gene'] == 'Tfdp2'])

     gene   32_degrees  32_degrees.1  32_degrees.2  32_degrees.3
40  Tfdp2  3349.830478   3247.058555   3030.374898   3307.762556
    gene   32_degrees  32_degrees.1  32_degrees.2  32_degrees.3
1  Tfdp2  3349.830478   3247.058555   3030.374898   3307.762556


# UMAP from louvain algorithm

In [14]:
# Save the file to .csv
df_mNPC_sp1rep = df_mNPC_sp1rep.set_index('gene')

df_mNPC_sp1rep.to_csv(file_directory+'1) input/from_output/df_mNPC_sp1rep.tsv',sep = "\t")
df_mNPC_sp1rep.head()

,32_degrees,32_degrees.1,32_degrees.2,32_degrees.3
gene,,,,
Creld2,751.602695,748.099155,778.773074,801.580112
Tfdp2,3349.830478,3247.058555,3030.374898,3307.762556
Noc3l,827.786899,853.633273,754.984167,665.868631
Pias3,1454.152781,1520.922170,1563.303656,1484.663052
Mns1,975.424977,783.570507,944.911361,746.454093


In [15]:
# Open the saved file with scanpy
adata  = scanpy.read_csv(file_directory+'1) input/from_output/df_mNPC_sp1rep.tsv', delimiter='\t')

#adata for anotated data
print(adata.shape)
adata

(267, 4)


AnnData object with n_obs × n_vars = 267 × 4

In [15]:
# yellow is group 0. Get all the gene in group 0
group_0_elements = adata.obs[adata.obs['louvain'] == '0'].index

print(f'genes in Smyd5 group: {group_0_elements}')

f = open(file_directory+'3) output/gene_in_smyd5_group.txt', 'w')
for element in group_0_elements:
  f.write('{}\n'.format(element))

f.close()

genes in Smyd5 group: Index(['Myo1a', 'Arhgap25', 'Cyp4x1', 'Tal2', 'Adra2b', 'Tyr', 'Rasd1', 'Bgn',
       'Rho', 'Col14a1', 'Cntnap2', 'B4galnt3', 'Kcnk12', 'Chmp4c', 'Scube2',
       'Grhl3', 'Tulp1', 'D130043K22Rik', 'Muc15', 'Dlec1', 'Nxnl1', 'Igf2bp1',
       'Calhm2', 'Prlr', 'Cd274', 'C1rl', 'Gjd4', 'Mapk15', 'Cyp2r1', 'Hhex',
       'Pon3'],
      dtype='object')


In [16]:
group_0_elements

Index(['Myo1a', 'Arhgap25', 'Cyp4x1', 'Tal2', 'Adra2b', 'Tyr', 'Rasd1', 'Bgn',
       'Rho', 'Col14a1', 'Cntnap2', 'B4galnt3', 'Kcnk12', 'Chmp4c', 'Scube2',
       'Grhl3', 'Tulp1', 'D130043K22Rik', 'Muc15', 'Dlec1', 'Nxnl1', 'Igf2bp1',
       'Calhm2', 'Prlr', 'Cd274', 'C1rl', 'Gjd4', 'Mapk15', 'Cyp2r1', 'Hhex',
       'Pon3'],
      dtype='object')